In [6]:
import os
import numpy as np
import pandas as pd

In [7]:
path_to_log_files_directory = "logs/"

In [8]:
def get_dir_paths(root_path):
    dir_paths = []
    
    for abs_path, dirs, files in os.walk(root_path):
        if len(dirs) == 0:
            dir_paths.append(os.path.relpath(abs_path, root_path) + "/")
    
    return dir_paths

In [9]:
def get_all_data(f_path):
    seconds = []
    throughputs = []
    line_cnt = 0
    
    with open(f_path, "r") as f:
        for line in f:
            data = line.rstrip().split()
            if line_cnt>50:
                break
                
            try:
                seconds.append(float(data[0]))
                throughputs.append(float(data[2]))
            except:
                pass
            
            line_cnt+=1
            
    while len(throughputs)>=2 and throughputs[0] == 0 and throughputs[1] == 0:
        del throughputs[0]
        
    while len(seconds) > len(throughputs):
        del seconds[-1]
        
    return seconds[:30], throughputs[:30]

In [10]:
df = pd.DataFrame({})

dir_paths = get_dir_paths(path_to_log_files_directory)
        
for i, path in enumerate(dir_paths):
    path = os.path.join(path_to_log_files_directory, path)
    dir_paths[i] = path
        
for dir_path in dir_paths:
    if 'freq' in dir_path.split('/'):
        continue
           
    files = [f for f in os.listdir(dir_path)]
            
    for file in files:
        try:
            if ".csv" in file:
                continue
                    
            file_path = os.path.join(dir_path, file)
            seconds, throughputs = get_all_data(file_path)
                
            if len(throughputs) == 0 or sum(throughputs) == 0:
                continue
            
            if 'pronghorn' in file_path:
                net_name = 'pronghorn'
            elif 'dtns' in file_path:
                net_name = 'dtns'
            elif 'esnet' in file_path:
                net_name = 'esnet'
            elif 'comet' in file_path or 'sc' in file_path:
                net_name = 'xsede'import log_file_reader
import dir_filter
            else:
                net_name = np.nan
            
            data = {
                "file_name": [file],
                "network_name": [net_name]
            }

            for i in range(len(throughputs)):
                key = 'p' + str(i+1)
                data[key] = [throughputs[i]]

            data['mean_throughput'] = [np.mean(throughputs)]
            data['median_throughput'] = [np.median(throughputs)]
            data['stdv_throughput'] = [np.std(throughputs)]
                    
            temp_df = pd.DataFrame(data)
            df = df.append(temp_df, ignore_index=True, sort=False)
        except Exception as e:
            file_path = os.path.join(dir_path, file)
            print("{0}: {1}".format(file_path, e))
            
df.to_csv("network_data_all.csv", index=False)